In [1]:
import asyncio
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List
from langchain_mcp_adapters.client import MultiServerMCPClient

# Set visualization style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Setup complete!")

Setup complete!


In [2]:
# Initialize MCP client for hosted server
async def init_mcp_client():
    """Initialize connection to hosted MCP server."""
    client = MultiServerMCPClient({
        "biodiversity-server": {
            "url": "https://biodiversity-mcp.nrp-nautilus.io/sse",
            "transport": "sse",
        }
    })
    tools = await client.get_tools()
    query_tool = next((t for t in tools if t.name == "query"), None)
    return query_tool

async def run_query(query: str) -> pd.DataFrame:
    """Execute a query via the MCP server and return as DataFrame."""
    query_tool = await init_mcp_client()
    if query_tool:
        result = await query_tool.ainvoke({"query": query})
        # The MCP server returns a formatted table string
        # We need to parse it or use a simpler format
        return result
    else:
        raise Exception("Query tool not found")

# Test connection with a simple query that returns JSON-parseable data
async def test_connection():
    try:
        # Test with a simple query
        query = """
        SET THREADS=100;
        INSTALL httpfs; LOAD httpfs;
        CREATE OR REPLACE SECRET s3 (
            TYPE S3,
            ENDPOINT 'rook-ceph-rgw-nautiluss3.rook',
            URL_STYLE 'path',
            USE_SSL 'false'
        );
        SELECT 'Connected' as status, 1 as test_value;
        """
        result = await run_query(query)
        print("✅ Connected to MCP server")
        print(f"Result type: {type(result)}")
        print(f"Result preview (first 500 chars): {str(result)[:500]}")
        return True
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return False

await test_connection()

✅ Connected to MCP server
Result type: <class 'str'>
Result preview (first 500 chars): +-----------+------------+
|  status   | test_value |
|  VARCHAR  |  INTEGER   |
+-----------+------------+
| Connected |     1      |
+-----------+------------+


True

In [ ]:
def parse_table_result(result_str: str) -> pd.DataFrame:
    """
    Parse ASCII table format from MCP query results into a DataFrame.
    
    Expected format:
    +------------+---------+
    |  column1   | column2 |
    |   TYPE1    |  TYPE2  |
    +------------+---------+
    | value1     | value2  |
    +------------+---------+
    """
    lines = result_str.strip().split('\n')
    
    if not lines or len(lines) < 3:
        return pd.DataFrame()
    
    # Find header line (first line after first separator)
    columns = []
    type_row_idx = -1
    
    for i, line in enumerate(lines):
        if line.startswith('+'):
            continue
        # First non-separator line is the header
        if not columns:
            columns = [c.strip() for c in line.split('|')[1:-1]]
            continue
        # Second non-separator line is the type row
        if any(dtype in line.upper() for dtype in ['BIGINT', 'INTEGER', 'DOUBLE', 'DECIMAL', 'FLOAT', 'VARCHAR', 'HUGEINT', 'BOOLEAN']):
            type_row_idx = i
            break
    
    if not columns or type_row_idx == -1:
        return pd.DataFrame()
    
    # Find data rows (after type row and next separator)
    data_rows = []
    in_data_section = False
    
    for i in range(type_row_idx + 1, len(lines)):
        line = lines[i]
        # Skip separator that comes right after type row
        if line.startswith('+'):
            in_data_section = True
            continue
        
        # Collect data rows
        if in_data_section and '|' in line and not line.startswith('+'):
            parts = [p.strip() for p in line.split('|')[1:-1]]
            if len(parts) == len(columns):
                data_rows.append(parts)
    
    # Create DataFrame
    if data_rows:
        df = pd.DataFrame(data_rows, columns=columns)
        # Try to convert numeric columns
        for col in df.columns:
            try:
                df[col] = pd.to_numeric(df[col])
            except (ValueError, TypeError):
                pass  # Keep as string if conversion fails
        return df
    else:
        return pd.DataFrame(columns=columns)

print("Table parser defined")

Table parser defined


## Analysis Function

The following function queries the biodiversity MCP server to analyze watersheds for a given country. It performs a multi-step SQL query that:

1. **Identifies wetland hexagons** within the country's watersheds (using spatial joins on H3 hexagons)
2. **Calculates watershed metrics** by aggregating data across multiple datasets
3. **Computes normalized scores** for each criterion
4. **Ranks watersheds** by composite score and returns the top N results

The function uses the Model Context Protocol (MCP) to execute DuckDB queries against cloud-hosted datasets.

In [4]:
async def analyze_hydrobasins_for_country(country_code: str, country_name: str, top_n: int = 3) -> pd.DataFrame:
    """
    Analyze Level 6 HydroBASINS for a given country using MCP server.
    
    Args:
        country_code: ISO 2-letter country code (e.g., 'US', 'CN')
        country_name: Full country name for display
        top_n: Number of top basins to return
        
    Returns:
        DataFrame with top hydrobasins and their scores
    """
    
    query = f"""
    SET THREADS=100;
    INSTALL httpfs; LOAD httpfs;
    CREATE OR REPLACE SECRET s3 (
        TYPE S3,
        ENDPOINT 'rook-ceph-rgw-nautiluss3.rook',
        URL_STYLE 'path',
        USE_SSL 'false'
    );
    
    WITH basin_wetlands AS (
        SELECT 
            hb.id as basin_id,
            hb.PFAF_ID,
            hb.UP_AREA,
            hb.SUB_AREA,
            w.h8,
            w.h0
        FROM read_parquet('s3://public-overturemaps/hex/countries.parquet') c
        JOIN read_parquet('s3://public-wetlands/glwd/hex/**') w 
            ON c.h8 = w.h8 AND c.h0 = w.h0
        JOIN read_parquet('s3://public-hydrobasins/level_06/hexes/**') hb 
            ON w.h8 = hb.h8 AND w.h0 = hb.h0
        WHERE c.country = '{country_code}' AND w.Z > 0
    ),
    basin_metrics AS (
        SELECT 
            bw.basin_id,
            bw.PFAF_ID,
            bw.UP_AREA,
            bw.SUB_AREA,
            COUNT(DISTINCT bw.h8) as wetland_hex_count,
            ROUND(COUNT(DISTINCT bw.h8) * 73.7327598, 2) as wetland_area_hectares,
            ROUND(COALESCE(SUM(carb.carbon), 0), 2) as total_carbon,
            ROUND(
                COUNT(DISTINCT CASE WHEN wdpa.h8 IS NOT NULL THEN bw.h8 END)::FLOAT / 
                NULLIF(COUNT(DISTINCT bw.h8), 0),
                3
            ) as protected_fraction,
            ROUND(AVG(ncp.ncp), 3) as avg_ncp_score
        FROM basin_wetlands bw
        LEFT JOIN read_parquet('s3://public-carbon/hex/vulnerable-carbon/**') carb 
            ON bw.h8 = carb.h8 AND bw.h0 = carb.h0
        LEFT JOIN read_parquet('s3://public-wdpa/hex/**') wdpa 
            ON bw.h8 = wdpa.h8 AND bw.h0 = wdpa.h0
        LEFT JOIN read_parquet('s3://public-ncp/hex/ncp_biod_nathab/**') ncp 
            ON bw.h8 = ncp.h8 AND bw.h0 = ncp.h0
        GROUP BY bw.basin_id, bw.PFAF_ID, bw.UP_AREA, bw.SUB_AREA
    )
    SELECT 
        basin_id,
        PFAF_ID,
        UP_AREA as upstream_area_km2,
        SUB_AREA as basin_area_km2,
        wetland_hex_count,
        wetland_area_hectares,
        total_carbon,
        protected_fraction,
        avg_ncp_score,
        ROUND(
            (wetland_area_hectares / MAX(wetland_area_hectares) OVER () * 0.25 +
             total_carbon / NULLIF(MAX(total_carbon) OVER (), 0) * 0.25 +
             protected_fraction * 0.25 +
             avg_ncp_score * 0.25),
            3
        ) as composite_score
    FROM basin_metrics
    WHERE wetland_hex_count > 0
    ORDER BY composite_score DESC
    LIMIT {top_n};
    """
    
    print(f"\n{'='*80}")
    print(f"Analyzing: {country_name} ({country_code})")
    print(f"{'='*80}")
    
    result_str = await run_query(query)
    df = parse_table_result(result_str)
    
    if len(df) > 0:
        df['country'] = country_name
        df['country_code'] = country_code
        
        print(f"\nTop {top_n} Priority Hydrobasins in {country_name}:")
        display_cols = ['basin_id', 'PFAF_ID', 'wetland_area_hectares', 'total_carbon', 
                       'protected_fraction', 'avg_ncp_score', 'composite_score']
        print(df[display_cols].to_string(index=False))
    else:
        print(f"\nNo wetland data found for {country_name}")
        df['country'] = country_name
        df['country_code'] = country_code
    
    return df

print("Analysis function defined")

Analysis function defined


## Priority Watershed Analysis Methodology

This analysis identifies priority watersheds for wetland conservation based on a composite score that integrates multiple conservation criteria. We use HydroBASINS Level 6 watersheds as our spatial units of analysis.

### Data Sources

The analysis integrates the following global datasets:

1. **Wetlands**: Global Lakes and Wetlands Database (GLWD) - provides wetland extent and type
2. **Carbon Storage**: Vulnerable carbon data from Conservation International - represents irrecoverable carbon stocks
3. **Protected Areas**: World Database on Protected Areas (WDPA) - identifies existing conservation coverage
4. **Biodiversity Value**: Nature's Contributions to People (NCP) scores - biodiversity and ecosystem service importance
5. **Watersheds**: HydroBASINS Level 6 - watershed boundaries for analysis units

### Composite Score Calculation

Each watershed receives a **composite score** (0-1 scale) based on four equally-weighted criteria:

#### 1. Wetland Area (25% weight)
- **Metric**: Total hectares of wetlands within the watershed
- **Normalization**: Scaled relative to the largest wetland area in the country
- **Rationale**: Larger wetland areas provide greater ecosystem services and habitat

#### 2. Vulnerable Carbon (25% weight)
- **Metric**: Total vulnerable carbon stocks (metric tons) in wetland areas
- **Normalization**: Scaled relative to the highest carbon stock in the country
- **Rationale**: Identifies wetlands whose loss would release significant greenhouse gases

#### 3. Protection Gap (25% weight)
- **Metric**: Fraction of wetland area NOT currently protected (1 - protected_fraction)
- **Range**: 0 (fully protected) to 1 (completely unprotected)
- **Rationale**: Prioritizes watersheds with low current protection status

#### 4. Biodiversity & Ecosystem Services (25% weight)
- **Metric**: Average Nature's Contributions to People (NCP) score
- **Range**: 0 (low importance) to 1 (high importance)
- **Rationale**: Identifies areas critical for biodiversity and human well-being

### Formula

```
composite_score = (
    (wetland_area / max_wetland_area) * 0.25 +
    (total_carbon / max_total_carbon) * 0.25 +
    (1 - protected_fraction) * 0.25 +
    avg_ncp_score * 0.25
)
```

### Interpretation

- **Higher scores** (closer to 1.0) indicate watersheds with:
  - Large wetland areas
  - High carbon storage
  - Low protection status (high conservation need)
  - High biodiversity/ecosystem service value
  
- **Lower scores** indicate watersheds that are either:
  - Already well-protected
  - Smaller wetland systems
  - Lower carbon/biodiversity values

### Important Notes

- **GLWD Multiple Categories**: A single hexagon in GLWD can have multiple wetland type codes. We use `COUNT(DISTINCT h8)` to avoid counting the same location multiple times.
- **H3 Hexagons**: All data is indexed using H3 hexagons at resolution 8, where each hex = 73.73 hectares
- **Country-Specific Normalization**: Scores are normalized within each country to identify relative priorities

## North America: United States, Canada, and Mexico

In [5]:
# United States
us_results = await analyze_hydrobasins_for_country('US', 'United States', top_n=3)


Analyzing: United States (US)

No wetland data found for United States


In [ ]:
# Canada
canada_results = await analyze_hydrobasins_for_country('CA', 'Canada', top_n=3)

In [ ]:
# Mexico
mexico_results = await analyze_hydrobasins_for_country('MX', 'Mexico', top_n=3)

## Asia: China, South Korea, and Thailand

In [ ]:
# China
china_results = await analyze_hydrobasins_for_country('CN', 'China', top_n=3)

In [ ]:
# South Korea
korea_results = await analyze_hydrobasins_for_country('KR', 'South Korea', top_n=3)

In [ ]:
# Thailand
thailand_results = await analyze_hydrobasins_for_country('TH', 'Thailand', top_n=3)

## Europe: United Kingdom, France, and Spain

In [ ]:
# United Kingdom
uk_results = await analyze_hydrobasins_for_country('GB', 'United Kingdom', top_n=3)

In [ ]:
# France
france_results = await analyze_hydrobasins_for_country('FR', 'France', top_n=3)

In [ ]:
# Spain
spain_results = await analyze_hydrobasins_for_country('ES', 'Spain', top_n=3)

## South America: Brazil and Chile

In [ ]:
# Brazil
brazil_results = await analyze_hydrobasins_for_country('BR', 'Brazil', top_n=3)

In [ ]:
# Chile
chile_results = await analyze_hydrobasins_for_country('CL', 'Chile', top_n=3)

## Australia

In [ ]:
# Australia
australia_results = await analyze_hydrobasins_for_country('AU', 'Australia', top_n=3)

## India

In [ ]:
# India
india_results = await analyze_hydrobasins_for_country('IN', 'India', top_n=3)

## Summary: Combined Results

In [ ]:
# Combine all results
all_results = pd.concat([
    us_results, canada_results, mexico_results,
    china_results, korea_results, thailand_results,
    uk_results, france_results, spain_results,
    brazil_results, chile_results,
    australia_results, india_results
], ignore_index=True)

print("\n" + "="*80)
print("SUMMARY: Top Priority Hydrobasins Across All Countries")
print("="*80)
print(all_results[[
    'country', 'basin_id', 'PFAF_ID', 'wetland_area_hectares', 
    'total_carbon', 'protected_fraction', 'avg_ncp_score', 'composite_score'
]].to_string(index=False))

# Save to CSV
all_results.to_csv('priority_hydrobasins_results_mcp.csv', index=False)
print("\nResults saved to: priority_hydrobasins_results_mcp.csv")

## Visualization: Comparative Analysis

In [ ]:
# Create comparison plots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# A. Wetland Area by Country
country_totals = all_results.groupby('country')['wetland_area_hectares'].sum().sort_values(ascending=False)
axes[0, 0].barh(country_totals.index, country_totals.values, color='steelblue')
axes[0, 0].set_xlabel('Total Wetland Area (hectares)')
axes[0, 0].set_title('A. Wetland Area in Top Hydrobasins by Country')
axes[0, 0].grid(axis='x', alpha=0.3)

# B. Carbon Storage by Country
carbon_totals = all_results.groupby('country')['total_carbon'].sum().sort_values(ascending=False)
axes[0, 1].barh(carbon_totals.index, carbon_totals.values, color='darkgreen')
axes[0, 1].set_xlabel('Total Vulnerable Carbon')
axes[0, 1].set_title('B. Carbon Storage in Top Hydrobasins by Country')
axes[0, 1].grid(axis='x', alpha=0.3)

# C. Protected Fraction by Country
protected_avg = all_results.groupby('country')['protected_fraction'].mean().sort_values(ascending=False)
axes[1, 0].barh(protected_avg.index, protected_avg.values, color='orange')
axes[1, 0].set_xlabel('Average Protected Fraction')
axes[1, 0].set_title('C. Protection Coverage in Top Hydrobasins by Country')
axes[1, 0].set_xlim(0, 1)
axes[1, 0].grid(axis='x', alpha=0.3)

# D. NCP Score by Country
ncp_avg = all_results.groupby('country')['avg_ncp_score'].mean().sort_values(ascending=False)
axes[1, 1].barh(ncp_avg.index, ncp_avg.values, color='purple')
axes[1, 1].set_xlabel('Average NCP Score')
axes[1, 1].set_title('D. Nature Contributions in Top Hydrobasins by Country')
axes[1, 1].set_xlim(0, 1)
axes[1, 1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('priority_hydrobasins_comparison_mcp.png', dpi=300, bbox_inches='tight')
plt.show()

print("Visualization saved to: priority_hydrobasins_comparison_mcp.png")

In [ ]:
# Composite score comparison
plt.figure(figsize=(14, 8))
sns.boxplot(data=all_results, x='country', y='composite_score', palette='Set2')
plt.xticks(rotation=45, ha='right')
plt.xlabel('Country')
plt.ylabel('Composite Score')
plt.title('Distribution of Composite Scores Across Countries (Top 3 Hydrobasins Each)')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('composite_score_distribution_mcp.png', dpi=300, bbox_inches='tight')
plt.show()

print("Composite score distribution saved to: composite_score_distribution_mcp.png")

## Key Findings

### Methodology

For each country, we identified the top 3 Level 6 HydroBASINS based on a composite score that equally weights four key metrics:

1. **Wetland Area (25%)**: Total hectares of wetlands from GLWD
2. **Carbon Storage (25%)**: Vulnerable carbon in wetlands
3. **Protection Status (25%)**: Fraction of wetlands within WDPA protected areas
4. **Nature's Contributions (25%)**: Average NCP biodiversity score

### Interpretation

The composite score helps identify hydrobasins that balance multiple conservation priorities:
- High wetland area indicates ecological significance
- High carbon storage suggests climate mitigation importance
- Low protection fraction highlights conservation gaps
- High NCP scores indicate biodiversity value and ecosystem services

### Next Steps

The results can be used to:
- Prioritize watersheds for conservation investment
- Identify protection gaps in high-value wetlands
- Support climate and biodiversity policy decisions
- Guide restoration and protection efforts